In [21]:
import torch
from ultralytics import YOLO

In [ ]:
# check if access to GPU is enabled
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

MPS available: True
MPS built: True


In [5]:
# loading pretrained yolov8nano
model = YOLO("yolov8n.pt")

In [ ]:
model.train(
    data="data.yaml",
    epochs=10,
    imgsz=640,
    device="mps",
    augment=False,
    val = False,
    mosaic = 0
)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0 MPS (Apple M3 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

/opt/miniconda3/envs/fp-enum/lib/python3.12/site-packages/ultralytics/utils/plotting.py:784: RuntimeWarning: invalid value encountered in cast
  for j, box in enumerate(boxes.astype(np.int64).tolist()):


       1/10      5.35G        nan        nan        nan        212        640: 100% ━━━━━━━━━━━━ 2/2 0.2it/s 11.8s28.8s

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/10       4.4G        nan        nan        nan        208        640: 100% ━━━━━━━━━━━━ 2/2 0.6it/s 3.2s4.6s

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/10      4.54G        nan        nan        nan        162        640: 100% ━━━━━━━━━━━━ 2/2 0.4it/s 5.1s7.9s

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       4/10      4.61G        nan        nan        nan        199        640: 100% ━━━━━━━━━━━━ 2/2 0.5it/s 4.3s7.7s

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       5/10      4.61G        nan        nan        nan        162        640: 100% ━━━━━━━━━━━━ 2/2 0.6it/s 3.3s4.6s

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       6/10  

IndexError: index 55205 is out of bounds for axis 1 with size 13

In [9]:
import os

label_dir = "data/labels/train"

bad_files = []
num_classes = 12  # you have 0–12

for fname in os.listdir(label_dir):
    if not fname.endswith(".txt"):
        continue
    with open(os.path.join(label_dir, fname)) as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls = int(float(parts[0]))  # first number is class
            if cls < 0 or cls >= num_classes:
                bad_files.append((fname, cls))

print("Problematic labels:", bad_files)


Problematic labels: []


In [11]:
import os

label_dirs = ["data/labels/train", "data/labels/val", "data/labels/test"]

bad_entries = []

for d in label_dirs:
    for fname in os.listdir(d):
        if not fname.endswith(".txt"):
            continue
        path = os.path.join(d, fname)
        with open(path) as f:
            for i, line in enumerate(f, start=1):
                parts = line.strip().split()
                if not parts:
                    continue
                try:
                    cls = int(float(parts[0]))
                except ValueError:
                    bad_entries.append((path, i, line.strip()))
                    continue
                if cls < 0:
                    bad_entries.append((path, i, line.strip()))

if bad_entries:
    print("❌ Found invalid class IDs:")
    for entry in bad_entries:
        print(entry)
else:
    print("✅ No negative class IDs found.")


✅ No negative class IDs found.


In [12]:
import os

label_dirs = ["data/labels/train", "data/labels/val", "data/labels/test"]

empty_files = []
for d in label_dirs:
    for f in os.listdir(d):
        if f.endswith(".txt"):
            path = os.path.join(d, f)
            if os.path.getsize(path) == 0:
                empty_files.append(path)

print("Empty label files:", empty_files)


Empty label files: []


In [13]:
bad_format = []
for d in label_dirs:
    for f in os.listdir(d):
        if not f.endswith(".txt"):
            continue
        path = os.path.join(d, f)
        with open(path) as file:
            for i, line in enumerate(file, 1):
                parts = line.strip().split()
                if len(parts) != 5:
                    bad_format.append((path, i, line.strip()))

print("Files with wrong format:", bad_format)


Files with wrong format: []


In [17]:
import os

label_dirs = ["data/labels/train", "data/labels/val", "data/labels/test"]
num_classes = 12  # 0–11

bad_labels = []

for d in label_dirs:
    for f in os.listdir(d):
        if not f.endswith(".txt"):
            continue
        path = os.path.join(d, f)
        with open(path) as file:
            for i, line in enumerate(file, 1):
                parts = line.strip().split()
                if not parts:
                    continue
                cls = int(float(parts[0]))
                if cls >= num_classes:
                    bad_labels.append((path, i, cls))

print("Labels with out-of-range class indices:", bad_labels)


Labels with out-of-range class indices: []


In [18]:
import os

# Paths to your label folders
label_dirs = ["data/labels/train", "data/labels/val", "data/labels/test"]

# Maximum valid class index
num_classes = 12  # 0–11

# Go through each label file
for d in label_dirs:
    for f in os.listdir(d):
        if not f.endswith(".txt"):
            continue
        path = os.path.join(d, f)
        # Read all lines
        with open(path, "r") as file:
            lines = file.readlines()
        
        # Keep only lines with valid class index
        new_lines = []
        for line in lines:
            if not line.strip():
                continue
            parts = line.strip().split()
            cls = int(float(parts[0]))
            if 0 <= cls < num_classes:
                new_lines.append(line)
            else:
                print(f"Removed invalid class {cls} in {path}")
        
        # Overwrite file with cleaned lines
        with open(path, "w") as file:
            file.writelines(new_lines)

print("✅ All labels cleaned.")


✅ All labels cleaned.
